In [11]:
!pip install -q youtube-transcript-api langchain-community langchain-openai \
               faiss-cpu tiktoken python-dotenv

In [63]:
from dotenv import load_dotenv 
load_dotenv()

True

In [65]:
from youtube_transcript_api import YouTubeTranscriptApi
from youtube_transcript_api._errors import TranscriptsDisabled

video_id = "8IU7YBgpQxg"

try:
    fetched_transcript = YouTubeTranscriptApi().fetch(video_id, languages=['en'])
    transcript_list = fetched_transcript.to_raw_data()
    transcript = " ".join(chunk["text"] for chunk in transcript_list)
    print(transcript)
    
except TranscriptsDisabled:
    print("No captions available for this video.")
except Exception as e:
    print(f"An error occurred: {e}")

in your eyes where is this warlike situation Israel Iran going to some extent it's a religious issue it's about Muslims of Jews and they are trying to create this pan Turan Corridor which will essentially Wipe Out the nation of Armenia and create a whole turkey Corridor this Iran thing came up the Gaza Israel thing is already going on what do you think the mood is like in that Nation Gaza has been sealed off Palestinians obviously want Israel to cease to exist but long term there is this deep hatred which is like a blood hatred what's been India's stand on both these conflicts India's borders are likely to change in some way in which direction I would not like to talk about it he said it already what's happening on a global scale Europe is in recession people are worried in Europe that Putin may have expansionist Ambitions and would want to take Poland Hungary there's a cold war happening in Africa as well drone Warfare is now an out andout reality it's gone from being something theore

In [19]:
transcript_list


[{'text': 'in your eyes where is this warlike',
  'start': 0.08,
  'duration': 4.64},
 {'text': 'situation Israel Iran going to some',
  'start': 2.36,
  'duration': 3.72},
 {'text': "extent it's a religious issue it's about",
  'start': 4.72,
  'duration': 3.039},
 {'text': 'Muslims of Jews and they are trying to',
  'start': 6.08,
  'duration': 3.639},
 {'text': 'create this pan Turan Corridor which',
  'start': 7.759,
  'duration': 3.88},
 {'text': 'will essentially Wipe Out the nation of',
  'start': 9.719,
  'duration': 3.321},
 {'text': 'Armenia and create a whole turkey',
  'start': 11.639,
  'duration': 3.681},
 {'text': 'Corridor this Iran thing came up the',
  'start': 13.04,
  'duration': 4.6},
 {'text': 'Gaza Israel thing is already going on',
  'start': 15.32,
  'duration': 3.56},
 {'text': 'what do you think the mood is like in',
  'start': 17.64,
  'duration': 3.16},
 {'text': 'that Nation Gaza has been sealed off',
  'start': 18.88,
  'duration': 4.479},
 {'text': 'Pale

In [67]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=150)
chunks = splitter.create_documents([transcript])

In [16]:
len(chunks)

88

In [68]:
# 3) Embeddings
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
   

c:\Users\kapoo\OneDrive\Desktop\Campusx\Youtube RAG\venv\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [69]:
# Vector DB
from langchain_community.vectorstores import FAISS
vector_store = FAISS.from_documents(chunks, embeddings)
retriever = vector_store.as_retriever(search_kwargs={"k": 4})

In [70]:
# 4) Prompt template
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate(
    template=(
        "Use ONLY the transcript context below to answer.\n\n"
        "Context:\n{context}\n\n"
        "Question: {question}\n\n"
        "Answer:"
    ),
    input_variables=["context", "question"]
)

In [71]:
# 5) RAG pipeline
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

def join_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

parallel_chain = RunnableParallel(
    {
        "context": retriever | RunnableLambda(join_docs),
        "question": RunnablePassthrough(),
    }
)

In [ ]:
# ✅ MUST use conversational model support
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
import os

llm = HuggingFaceEndpoint(
    repo_id="Qwen/Qwen2.5-1.5B-Instruct",
    task="text-generation",
    max_new_tokens=200,
    huggingface_api_key=os.getenv("HUGGINGFACE_API_KEY")
)
chat_model = ChatHuggingFace(llm=llm)




WARNING! huggingface_api_key is not default parameter.
                    huggingface_api_key was transferred to model_kwargs.
                    Please make sure that huggingface_api_key is what you intended.


In [77]:
# 6) RAG chain
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

parallel_chain = RunnableParallel({
    "context": retriever | RunnableLambda(format_docs),
    "question": RunnablePassthrough(),
})
parser = StrOutputParser()
rag_chain = parallel_chain | prompt | chat_model | parser

In [ ]:
# Query
response = rag_chain.invoke("Summarize the video.")
print("\nRAG Response:\n", response)